In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from skimage.feature import hog
import skimage.io as io
from skimage.util import view_as_blocks
import pandas as pd
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression, SGDClassifier, LogisticRegressionCV
import pickle
import numpy as np
import warnings
import math

In [2]:
# Fit encoders in order to map string labels to integers
# The fact that we are storing the scikit objects makes the encoding reproducible and consistent
# We do not need to fit each time, we store the fitted object and use its transform method when needed

label_encoder_specific = LabelEncoder().fit(pd.read_csv('dataset_cards/cards.csv')['labels'])  # Specific
label_encoder_category = LabelEncoder().fit(pd.read_csv('dataset_cards/cards.csv')['card type'])  # card type

### HOG

In [3]:
from skimage.feature import hog
from skimage import color

In [61]:
class CardsDataset(Dataset):
    """
    A PyTorch Dataset class specifically aimed at digesting and retrieving the images of the cards, together with their
    labels, which are also reliably encoded.
    """

    def __init__(self, csv_path: str, root_dir: str, transform: str,
                 card_category: bool, label_encoder: LabelEncoder,
                 subset: str = 'train') -> None:
        """
        Initialization of the CardDataset class
        :param csv_path: The path of the csv where to retrieve the necessary information about the images of the cards.
        :param root_dir: The root directory for the dataset.
        :param transform: Option between Histogram of Gradients ('hog') or Histogram of colors ('rgb_hist').
        Pass None in order to avoid transformations (normalization at pixel level is the only operation performed).
        :param card_category: Whether the label refers to the card category or to the specific card itself.
        :param label_encoder: The scikit-learn LabelEncoder object necessary to map the string labels to integers.
        :param subset: Choose between train ('train'), test ('test') and validation ('valid') dataset.
        """
        total_cards = pd.read_csv(csv_path)
        try:
            self.cards_table = total_cards[total_cards['data set'] == subset]
        except KeyError:
            raise KeyError('There is no subset in the dataset with the queried label')
        with warnings.catch_warnings():
            warnings.simplefilter('ignore')
            self.cards_table['labels' if not card_category else 'card type'] = \
                label_encoder.transform(self.cards_table['labels']) if not card_category else \
                label_encoder.transform(self.cards_table['card type'])
        self.root_dir = root_dir
        self.transform = transform
        self.card_category = card_category

    def __len__(self):
        return len(self.cards_table)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        card_class = self.cards_table.iat[idx, 2] if not self.card_category else self.cards_table.iat[idx, 3]

        if self.transform == 'hog':
            representation = np.load(os.path.join(self.root_dir, 'hog',
                                                  self.cards_table.iat[idx, 1].split('.')[0] + '.npy'))
            representation = np.moveaxis(representation, 2, 0)
            
        elif self.transform == 'hog_2':
            img_path = os.path.join(self.root_dir, self.cards_table.iat[idx, 1])
            image = color.rgb2gray(io.imread(img_path))
            fd, hog_image = hog(image, orientations=6, pixels_per_cell=(22, 22),
                    cells_per_block=(1, 1), block_norm="L1", visualize=True, feature_vector = True)
            representation = fd
            
        elif self.transform is None or self.transform == 'rgb_hist':
            img_path = os.path.join(self.root_dir, self.cards_table.iat[idx, 1])
            image = io.imread(img_path)
            if self.transform == 'rgb_hist':
                representation, _ = np.histogramdd((np.reshape(image, (-1, 3))), bins=[np.linspace(0, 256, 6)] * 3)
                representation = (representation / representation.sum()).flatten()
            else:
                representation = np.moveaxis(image/255.0, 2, 0)

        else:
            raise ValueError(f'There is no transformation function for \'{self.transform}\'')

        return representation.astype(np.float32), card_class

### Training set

In [ ]:
training_rgb = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                                transform='rgb_hist', card_category=False,
                                label_encoder=label_encoder_specific, subset='train')

training_hog = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                            transform='hog_2', card_category=False,
                            label_encoder=label_encoder_specific, subset='train')

X_rgb = np.array([i[0] for i in training_rgb])
y_rgb = np.array([i[1] for i in training_rgb])
X_hog = np.array([i[0] for i in training_hog])
y_hog = np.array([i[1] for i in training_hog])

In [71]:
print(X_rgb.shape)
print(y_rgb.shape)

(7624, 125)
(7624,)


### Test set

In [10]:
test_rgb = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                                transform='rgb_hist', card_category=False,
                                label_encoder=label_encoder_specific, subset='test')

test_hog = CardsDataset(csv_path=os.path.join('dataset_cards', 'cards.csv'), root_dir='dataset_cards',
                        transform='hog_2', card_category=False,
                        label_encoder=label_encoder_specific, subset='test')
    
X_t_rgb = np.array([i[0] for i in test_rgb])
y_t_rgb = np.array([i[1] for i in test_rgb])
X_t_hog = np.array([i[0] for i in test_hog])
y_t_hog = np.array([i[1] for i in test_hog])

In [77]:
print(X_t_hog.shape)
print(y_t_hog.shape)

(265, 600)
(265,)


### model

In [73]:
clf = LogisticRegression(multi_class="multinomial", n_jobs=os.cpu_count()//2, solver="sag", verbose=1).fit(X_hog, y_hog)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 114 seconds


C:\Users\iadig\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  1.9min finished


In [78]:
print('Accuracy:', round(clf.score(X_hog, y_hog), 4))

Accuracy: 0.9117


In [85]:
# RISULTATO MIGLIORE
print('Accuracy:', round(clf.score(X_t_hog, y_t_hog), 4))

Accuracy: 0.6566


In [76]:
y_pred = clf.predict(X_t_hog)
print((y_pred == y_t).sum())
print(y_pred == y_t)

174
[False False False  True False False  True  True  True  True  True  True
  True  True  True False  True  True  True  True  True  True  True  True
  True  True  True False  True False  True False  True  True  True  True
 False False False  True  True False False False False  True  True  True
  True False  True False  True  True  True False False False False  True
 False False  True  True  True  True  True  True  True False  True  True
  True  True  True  True  True False False  True  True False  True  True
  True  True  True False  True False  True False  True  True  True  True
  True False  True  True False False False False False  True  True  True
  True False  True  True  True False  True False False  True  True  True
 False False  True  True False False  True  True False False  True  True
 False  True  True  True  True False False  True  True False  True  True
  True False  True  True False  True  True  True False  True  True  True
  True False False  True  True  True  True  Tru

### HOC + HOG

In [ ]:
X_rgb = np.load('X_rgb.npy')
X_hog = np.load('X_hog.npy')
y_rgb = np.load('y_rgb.npy')
y_hog = np.load('y_hog.npy')

In [65]:
X_rgb_hog = np.hstack((X_rgb, X_hog))
y_rgb_hog = y_hog

In [67]:
X_rgb_hog_t = np.hstack((X_t_rgb, X_t_hog))
y_rgb_hog_t = y_t_hog

In [68]:
clf_rgb_hog = LogisticRegression(multi_class="multinomial", n_jobs=os.cpu_count()//2, solver="sag", verbose=1).fit(X_rgb_hog, y_rgb_hog)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 139 seconds


C:\Users\iadig\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  2.3min finished


In [69]:
# test set accuracy
print('Accuracy:', round(clf_rgb_hog.score(X_rgb_hog_t, y_rgb_hog_t), 4))

Accuracy: 0.6491


In [70]:
# training set accuracy
print('Accuracy:', round(clf_rgb_hog.score(X_rgb_hog, y_rgb_hog), 4))

Accuracy: 0.9178


### Try normalizing X_rgb_hog and X_rgb_hog_t

In [80]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_rgb_hog)
scaled_X = scaler.transform(X_rgb_hog)
scaler.fit(X_rgb_hog_t)
scaled_X_t = scaler.transform(X_rgb_hog_t)

In [82]:
clf_rgb_hog_scaled = LogisticRegression(multi_class="multinomial", n_jobs=os.cpu_count()//2, solver="sag", verbose=1).fit(scaled_X, y_rgb_hog)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 139 seconds


C:\Users\iadig\anaconda\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[Parallel(n_jobs=4)]: Done   1 out of   1 | elapsed:  2.3min finished


In [84]:
# training set accuracy
print('Accuracy:', round(clf_rgb_hog_scaled.score(scaled_X_t, y_rgb_hog_t), 4))

Accuracy: 0.6113
